# # Dependencies and Loading Mistral

In [1]:
!pip install -q -U transformers

In [2]:
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

In [3]:
!pip install -Uqqq pip --progress-bar off


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
from transformers import (
    GenerationConfig,
    TextStreamer,
    pipeline,
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
model_name = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
from transformers import (
    GenerationConfig,
    TextStreamer,
    pipeline
)

generation_config = GenerationConfig.from_pretrained(model_name)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.do_sample = True

In [7]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [8]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    streamer=streamer,
    device_map="auto", 
    torch_dtype=torch.bfloat16, 
)
     

In [12]:
prompt = "[INST] Give one named recipe with steps using ingredients: egg whites, new potatoes, kosher salt, finely ground black pepper, rosemary, thyme, parsley. Give source url. [/INST]"
result = llm(prompt)

Here is a recipe for Roasted New Potatoes with Rosemary, Thyme, and Parsley:

Ingredients:

* 1 pound new potatoes, washed and cut into bite-sized pieces
* 4 egg whites
* 2 tablespoons kosher salt
* 1 teaspoon finely ground black pepper
* 2 tablespoons chopped fresh rosemary
* 2 tablespoons chopped fresh thyme
* 2 tablespoons chopped fresh parsley

Instructions:

1. Preheat the oven to 425°F (220°C).
2. In a large bowl, toss the new potatoes with the egg whites, kosher salt, and black pepper until the potatoes are evenly coated.
3. Spread the potatoes out in a single layer on a baking sheet lined with parchment paper.
4. Roast the potatoes in the oven for 20-25 minutes, or until they are tender and golden brown.
5. Remove the potatoes from the oven and sprinkle them with the chopped rosemary, thyme, and parsley. Toss to coat.
6. Return the potatoes to the oven and roast for an additional 5-10 minutes, or until the herbs are fragrant and the potatoes are crispy.
7. Serve the roasted new

# **Prompt Engineering**

In [9]:
list=[['whole chicken', 'kosher salt', 'acorn squash', 'sage', 'rosemary', 'unsalted butter', 'ground allspice', 'crushed red pepper flakes', 'freshly ground black pepper', 'white bread', 'apples', 'extra-virgin olive oil', 'red onion', 'apple cider vinegar', 'white miso', 'all-purpose flour', 'unsalted butter', 'dry white wine', 'unsalted chicken broth', 'white miso', 'kosher salt', 'freshly ground pepper'],
['egg whites', 'new potatoes', 'kosher salt', 'finely ground black pepper', 'rosemary', 'thyme', 'parsley'],
['evaporated milk', 'whole milk', 'garlic powder', 'onion powder', 'smoked paprika', 'freshly ground black pepper', 'kosher salt', 'extra-sharp cheddar', 'full-fat cream cheese', 'elbow macaroni'],
['round italian', 'olive oil', 'sweet italian sausage', 'unsalted butter', 'onions', 'celery', 'garlic', 'eggs', 'heavy cream', 'turkey giblet stock or reduced-sodium chicken broth', 'parmigiano-reggiano', 'flat-leaf parsley', 'qt shallow ceramic or glass baking dish'],
['dark brown sugar', 'hot water', 'bourbon', 'fresh lemon juice', 'apple butter', 'garnish : orange twist and, or ground cinnamon'],
['chamomile tea bags', 'reposado tequila', 'fresh lemon juice', 'agave nectar'],
['grand marnier', 'amaro averna', 'pat salted butter', 'hot apple cider', 'fresh lemon juice', 'garnish : freshly ground pink peppercorns'],
['granulated sugar', 'ground turmeric', 'amontillado sherry', 'bourbon, scotch', 'turmeric syrup', 'fresh lemon juice', 'garnish : dehydrated lemon wheel'],
['assorted dals', 'white jasmine rice or other long-grain rice', 'pearl barley', 'bone-in lamb stew meat', 'kosher salt', 'fresh ginger', 'shallots', 'ghee or vegetable oil', 'garlic', 'garam masala', 'cayenne pepper', 'ground turmeric', 'green thai chiles', 'cilantro', 'white onion', 'limes'],
['basic lentil soup', 'onion', 'turmeric', 'cumin', 'aleppo pepper or, red pepper flakes', 'tomato paste', 'eggs', 'ghee, unsalted butter', 'whole cumin seeds', 'olive oil', 'kosher salt and freshly ground black pepper', 'parsley'],
['chipotle in adobo sauce', 'garlic', 'kosher salt', 'cornstarch', 'evaporated milk', 'sharp or extra-sharp cheddar cheese', 'cream cheese', 'mayonnaise', 'pimento peppers', 'chipotle pepper purée'],
['bone-in', 'parsnips', 'carrots', 'maple syrup', 'unsweetened apple juice', 'dark brown sugar', 'smooth dijon mustard', 'grainy dijon mustard', 'apple cider vinegar', 'kosher salt'],
['bacon', 'celery', 'carrot', 'onion', 'cinnamon', 'boiling potatoes', 'granny smith apples', 'butternut squash', 'reduced-sodium chicken stock or broth', 'water', 'sour cream'],
['anchovies', 'garlic', 'mayonnaise', 'dijon mustard', 'extra-virgin olive oil', 'freshly ground black pepper', 'whole chicken', 'kosher salt', 'shallots', 'lemons', 'parmesan', 'country-style bread', 'romaine hearts'],
['skinless, boneless chicken thighs', 'kosher salt, freshly ground pepper', 'unsalted butter', 'leeks', 'zest and, lemon', 'long-grain white rice', 'low-sodium chicken broth', 'oil-packed anchovy fillet', 'garlic', 'capers', 'red pepper flakes', 'tender herb leaves', 'extra-virgin olive oil'],
['instant corn masa flour', 'lard, vegetable oil', 'diamond crystal or, morton kosher salt', 'guajillo or dried new mexico chiles', 'morita chiles', 'ground coriander or cumin', 'medium shrimp', 'garlic', 'thyme', 'apple cider vinegar', 'agave syrup or honey', 'unsalted butter', 'onion', 'avocado', 'cilantro leaves', 'and lime wedges'],
['vegetable oil', 'corn tortillas', 'fresh chorizo', 'garlic', 'white onion', 'kosher salt', 'black beans', 'low-sodium chicken broth', 'queso fresco or cotija cheese', 'cilantro leaves, avocado'],
['virgin coconut oil', 'ripe, plátanos manzanos or, ripe (black) plantains', 'kosher salt', 'plain unsweetened coconut milk yogurt or greek yogurt', 'ripe mangoes', 'toasted nuts', 'seeds', 'and / or unsweetened, coconut'],
['skin-on', 'kosher salt', 'potatoes', 'shallots', 'unsalted butter', 'heavy cream', 'garlic', 'freshly ground black pepper', 'or ground nutmeg', 'thyme', 'sage leaves', 'country-style bread', 'lemon'],
['of zest no pith, orange', 'fresh ginger', 'fresh lime juice', 'champagne vinegar', 'sugar', 'kosher salt', 'extra-virgin olive oil', 'thai chile', 'beets', 'extra-virgin olive oil', 'white wine vinegar', 'water', 'kosher salt and freshly ground black pepper', 'extra-virgin olive oil', 'cashews', 'sunchokes', 'vegetable oil', 'kosher salt', 'orange', 'roasted beets', 'extra-virgin olive oil', 'kosher salt', 'pickled orange-ginger puree', 'cashews', 'tarragon', 'dill fronds', 'sunchoke chips'],
['toasted pepitas', 'kale', 'garlic', 'chickpeas', 'ground turmeric', 'coriander (cilantro) leaves', 'sea salt and cracked black pepper', 'pumpkin', 'extra virgin olive oil', 'plain greek-style yogurt and hulled tahini', 'carrots', 'apple cider vinegar', 'honey', 'red chiles', 'coriander (cilantro) leaves'],
['butternut squash', 'extra virgin olive oil', 'pure maple syrup', 'chile flakes', 'sea salt and cracked black pepper', 'almonds', 'quinoa', 'mint leaves', 'wild rocket (arugula) leaves', 'hulled tahini', 'lemon juice', 'water', 'garlic', 'sea salt flakes'],
['ghee', 'paneer', 'confectioners’ sugar', 'ground cardamom', 'semolina', 'ghee', 'whole milk', 'cashews', 'golden raisins', 'granulated sugar', 'freshly whipped cream, almonds'],
['whole milk', 'lemons', 'kosher salt'],
['carrot', 'tomatoes', 'extra virgin olive oil', 'garlic', 'fresh spinach', 'kosher salt', 'ground turkey', 'ground cumin', 'smoked paprika', 'ground black pepper', 'r-rated onions', 'red wine vinegar', 'eggs parmigiano reggiano'],
['yellow or white onions', 'olive or vegetable oil', 'kosher salt'],
['extra-virgin olive oil', 'white miso', 'pure maple syrup or brown sugar', 'regular soy sauce or tamari', 'kabocha squash', 'extra-virgin olive oil', 'ginger', 'garlic', 'low-sodium vegetable broth', 'white miso', 'kosher salt, freshly ground pepper', 'broccolini', 'wavy ramen noodles', 'cilantro leaves with tender stems'],
['coconut oil', 'calabaza pumpkin', 'onion', 'garlic', 'ají dulce pepper', 'dark brown sugar', 'ground cumin', 'coconut milk', 'chicken broth or vegetable broth', 'salt and freshly ground black pepper', 'fresh cilantro'],
['italian eggplants', 'canola oil', 'onion', 'garlic', 'tomato', 'coarse salt', 'freshly ground black pepper', 'cilantro', 'roti'],
['chicken', 'lime juice', 'green seasoning', 'coarse salt', 'scotch bonnet pepper or other hot red chili pepper', 'garlic', 'trinidad curry powder', 'canola oil', 'onion', 'chicken stock or water', 'roti or white rice'],
['cardamom pods', 'coriander seeds', 'cumin seeds', 'mustard seeds', 'fenugreek seeds', 'curry leaves', 'whole black peppercorns', 'whole cloves', 'ground turmeric'],
['garlic', 'onions', 'celery', 'red bell pepper', 'green bell pepper', 'olive oil', 'chicken broth', 'whole tomatoes', 'bay leaf', 'ground cumin', 'salt', 'freshly ground black pepper', 'large shrimp', 'accompaniment, uncooked rice', 'prepared according'],
['fresh chives', 'fresh shado beni or cilantro leaves', 'fresh thyme', 'fresh oregano', 'fresh parsley', 'garlic'],
['dry or 1, pigeon peas, pinto beans', 'long-grain rice', 'canola oil', 'sugar', 'chicken', 'onion', 'garlic', 'coconut milk', 'bay leaf', 'green seasoning', 'parsley', 'thyme', 'carrots', 'scallions', 'kosher salt', 'fresh calabaza or butternut squash', 'whole scotch bonnet pepper', 'ketchup', 'butter'],
['yukon gold potatoes', 'garlic', 'kosher salt', 'whole milk', 'thyme', 'unsalted butter', 'freshly ground black pepper', 'sour cream', 'a potato ricer or food mill'],
['london dry gin', 'cynar', 'fino sherry', 'orange twist'],
['graham cracker crumbs', 'granulated sugar', 'light brown sugar', 'kosher salt', 'unsalted butter', 'cream cheese', 'granulated sugar', 'creole cream cheese', 'vanilla bean paste', 'eggs', 'ingredient info, sour cream mixed with, buttermilk, creole cream cheese.', 'unsalted butter', 'granulated sugar', 'light brown sugar', 'honey', 'granny smith apples', 'lemon zest', 'fresh lemon juice', 'kosher salt', 'ground cinnamon', 'ground cardamom', 'ground ginger', 'vanilla extract'],
['skim milk', 'buttermilk', 'rennet tablet'],
['nonfat dry milk powder', 'baking powder', 'granulated sugar or light brown sugar', 'all-purpose flour', 'vegetable oil', 'vegetable shortening', 'ghee', 'unsalted butter', 'butter'],
['eggs', 'mayonnaise', 'dijon mustard', 'cayenne', 'garnishes : paprika and, fresh chives', 'pastry, star tip'],
['green, unripe mangoes', 'shallots', 'indian or thai green chiles', 'ginger', 'garlic', 'coconut vinegar or white vinegar', 'salt', 'coconut milk', 'coconut oil', 'black mustard seeds', 'dried red chiles', 'curry leaves', 'fried onions', 'basmati rice'],
['piloncillo or, dark brown sugar', 'cardamom pods', 'allspice berries', 'black peppercorns', 'kosher salt', 'orange zest', 'cold-brew coffee', 'unsweetened coconut milk or half-and-half'],
['eggs', 'pumpkin purée', 'milk', 'unsalted butter', 'all-purpose flour', 'cinnamon', 'allspice', 'ground ginger', 'nutmeg', 'kosher salt', 'apple cider or juice', 'maple syrup', 'powdered sugar'],
['chicken legs', 'kosher salt', 'extra-virgin olive oil', 'fennel', 'red flame or green table grapes', 'unsalted chicken broth', 'honey', 'calabrian chile paste or, calabrian chiles', 'red wine vinegar'],
['onion', 'unsalted butter', 'all-purpose flour', 'milk', 'boiling potatoes', 'sharp cheddar', 'dry bread crumbs'],
['skinless salmon fillet', 'greek yogurt', 'garlic granules', 'rose harissa', 'ground turmeric', 'paprika', 'zest of, unwaxed lime and', 'olive oil', 'maldon sea salt flakes and freshly ground black pepper', 'tortilla wraps', 'sliced tomatoes', 'onion', 'coriander leaves', 'greek yogurt'],
['water', 'instant yeast', 'bread flour', 'canola oil', 'morton kosher salt'],
['beets', 'white vinegar', 'boneless pork butt', 'beef, chicken', 'bay leaf', 'whole allspice berries', 'whole black peppercorns', 'carrots', 'celery', 'green cabbage', 'idaho potatoes', 'lima beans', 'salt'],
['skin-on red snapper fillets', 'okra salt or kosher salt', 'alligator pepper or freshly ground black pepper', 'cubeb pepper or ashanti pepper', 'green or medium-ripe papaya', 'cilantro', 'extra-virgin olive oil', 'red onion', 'garlic', 'ginger', 'ground dried bird chiles or other hot chile powder', 'lemon', 'puna, white yams, sweet potatoes', 'coconut sugar or dark brown sugar', 'kosher salt', 'extra-virgin olive oil', 'freshly ground black pepper', 'chile powder', 'drunk apricot shito', 'a spice mill or mortar and pestle'],
['dried apricots', 'dark rum or arrack', 'unsalted butter', 'ground cinnamon', 'coconut sugar or dark brown sugar', 'grains of selim pods', 'cubeb pepper or ashanti pepper', 'peanut oil or vegetable oil', 'white onion', 'red onion', 'garlic', 'ginger', 'red pepper flakes', 'rosemary', 'thyme', 'dried prawn or shrimp powder', 'okra salt or kosher salt', 'tomato purée', 'dried chile powder', 'low-sodium chicken broth', 'scotch bonnet or habanero chile', 'a spice mill or mortar and pestle'],
['dried beans', 'dried kombu', 'tomatoes', 'celery', 'carrot', 'leek', 'shallots', 'garlic', 'oil-packed anchovy fillets', 'extra-virgin olive oil', 'maple syrup', 'kosher salt', 'red pepper flakes', 'whole-grain mustard', 'bacon', 'romano beans or green beans', 'shallots', 'red pepper flakes', 'zest and, lemon', 'garlic', 'pickled green beans', 'cherry tomatoes', 'extra-virgin olive oil', 'sherry vinegar', 'tomato brown butter', 'feta', 'parsley leaves', 'and marjoram leaves'],
['tomato', 'unsalted butter', 'kosher salt', 'freshly ground black pepper'],
['acorn squash', 'unsalted butter', 'pure maple syrup', 'good olive oil', 'kosher salt and freshly ground black pepper', 'flaked sea salt'],
['extra-virgin olive oil', 'serrano chile or jalapeño', 'fresh ginger', 'garlic', 'scallions', 'ground turmeric', 'corn', 'grains', 'unsweetened coconut milk', 'kosher salt', 'store-bought crispy onions or shallots', 'lime wedges'],
['mixed heirloom tomatoes', 'extra-virgin olive oil', 'kosher salt', 'freshly ground black pepper', 'garlic', 'unsalted butter', 'stoned wheat thins or other whole grain crackers', 'eggs', 'taleggio cheese', 'parmesan', 'mayonnaise', 'thyme', 'shallot', 'special equipment, springform pan'],
['scallions', 'ginger', 'garlic', 'coriander seeds', 'cumin seeds', 'black mustard seeds', 'kashmiri chile powder or, cayenne powder', 'ground turmeric', 'extra-virgin olive oil', 'paneer, planks', 'kosher salt', 'cherry tomatoes', 'sugar snap peas', 'sugar', 'mint leaves', 'rice'],
['skinless, boneless chicken breasts', 'kosher salt', 'scallions', 'persian cucumbers', 'sesame seeds', 'fresh lime juice', 'tahini', 'soy sauce', 'honey', 'red pepper flakes', 'romaine or green or red leaf lettuce, little gem lettuce', 'cilantro leaves with tender stems', 'lime'],
['extra-virgin olive oil', 'garlic', 'double-concentrated tomato paste', 'pure maple syrup', 'soy sauce', 'worcestershire sauce', 'unseasoned rice vinegar', 'sriracha or other hot sauce', 'vegetable oil', 'whole chicken', 'kosher salt'],
['egg yolks', 'kefir (cultured milk) or plain whole-milk yogurt', 'dill fronds with tender stems', 'garlic', 'unsalted butter', 'pine nuts, almonds', 'dried currants or raisins', 'kosher salt', 'ground cumin', 'freshly ground black pepper', 'ground lamb', 'orecchiette or other short pasta', 'lemon'],
['short-grain sushi rice', 'scallions', 'ginger', 'vegetable oil', 'skinless, boneless cod', 'kosher salt', 'freshly ground black pepper', 'unseasoned rice vinegar', 'sugar', 'unsalted butter'],
['boneless, skinless center-cut salmon', 'scallions', 'ginger', 'garlic', 'mayonnaise', 'kosher salt', 'sesame oil', 'unseasoned rice vinegar', 'persian cucumbers', 'serrano chile', 'sugar', 'vegetable oil', 'rice flour', 'tender herbs', 'watercress', 'sesame seeds', 'brioche buns'],
['guajillo chiles', 'morita chiles', 'skinless striped bass or halibut fillets', 'fresh orange juice', 'achiote paste', 'garlic, whole', 'fresh lime juice', 'kosher salt', 'red onion', 'habanero chile', 'pineapple', 'cilantro', 'vegetable oil', 'corn tortillas', 'lime wedges'],
['raw nuts', 'raw pumpkin seeds', 'extra-virgin olive oil', 'old-fashioned oats', 'dried fruit', 'unsalted or low-salt natural peanut butter', 'honey', 'kosher salt', 'vanilla extract', 'egg white'],
['whole-milk yogurt', 'fresh lemon juice', 'sugar', 'kosher salt', 'red lentils', 'zucchini', 'onion', 'kosher salt', 'kashmiri chile powder or, cayenne pepper', 'ground turmeric', 'parsley leaves', 'lemon zest', 'ghee or neutral vegetable oil'],
['vegetable oil', 'cooking onions'],
['baby yukon gold potatoes', 'kosher salt', 'sour cream', 'mayonnaise', 'onion powder', 'dijon mustard', 'freshly ground black pepper', 'garlic', 'chives', 'red onion', 'sour cream and onion potato chips'],
['ground coriander', 'ground cumin', 'ground turmeric', 'red pepper flakes', 'extra-virgin olive oil', 'garlic', 'skinless, boneless chicken thighs', 'kosher salt', 'fresh lime juice', 'ripe avocados', 'ripe mangoes', 'red onion', 'torn little gem or green lettuce leaves', 'cherry tomatoes', 'flaky sea salt', 'mild red pepper flakes', 'and basil and / or mint leaves'],
['golden raisins', 'thin-skinned cucumbers', 'kosher salt', 'extra-virgin olive oil', 'pine nuts', 'castelvetrano olives', 'egg', 'garlic', 'panko', 'parsley', 'red pepper flakes', 'freshly ground black pepper', 'ground pork', 'red onion', 'tender herb leaves', 'red wine vinegar', 'plain whole-milk greek yogurt'],
['butter', 'whole graham crackers', 'digestive biscuits', 'sweetened condensed milk', 'hass avocados', 'cream cheese', 'fine sea salt', 'lemons', 'limes'],
['all purpose flour', 'unsweetened cocoa powder', 'baking soda', 'salt', 'sugar', 'unsalted butter', 'vegetable oil', 'eggs', 'vanilla extract', 'buttermilk', 'unpeeled zucchini', 'semisweet chocolate chips', 'walnuts'],
['hazelnuts', 'bow tie pasta', 'unsalted butter', 'garlic', 'salt and freshly ground black pepper', 'crushed red pepper flakes', 'swiss chard', 'parmesan cheese', 'balsamic vinegar'],
['olive oil', 'garlic', 'cherry tomatoes', 'dry white wine', 'fresh basil leaves', 'fine sea salt', 'freshly ground black pepper', 'cooking fat', 'scallions', 'garlic', 'potato', 'fresh, frozen corn kernels', 'eggs', 'red pepper flakes', 'feta cheese'],
['all-purpose flour', 'salt', 'dried parsley', 'unsalted butter', 'sour cream or plain full-fat yogurt', 'white wine vinegar', 'ice water', 'cooking fat', 'onion', 'mixed heirloom tomatoes', 'hard salty cheese', 'mozzarella cheese', 'mayonnaise', 'basil leaves', 'parsley', 'thyme', 'salt and freshly ground black pepper', 'egg', 'milk', 'cream', 'or water'],
['" bone-in pork chops', 'kosher salt', 'coriander seeds', 'small padrón chiles or shishito peppers', 'red onion', 'garlic', 'carrots', 'red wine vinegar', 'sherry vinegar or red wine vinegar', 'honey', 'paprika', 'extra-virgin olive oil', 'oregano'],
['dashi', 'doenjang', 'gochujang', 'gochugaru', 'garlic', 'soy sauce', 'onions', 'green zucchini', 'red korean chili pepper or other long hot chili', 'green korean chili pepper or other long hot chili', 'brisket', 'soft or silken tofu', 'scallions'],
['chicken wingettes, chicken', 'kosher salt', 'ground black pepper', 'potato starch', 'vegetable oil', 'peanuts', 'rice syrup or honey', 'soy sauce', 'brown or white sugar', 'white vinegar', 'yellow mustard', 'vegetable oil', 'garlic', 'peeled ginger', 'dried red chili peppers', 'sesame seeds', 'crushed red pepper flakes'],
['onion', 'gochujang', 'soy sauce', 'garlic', 'sugar', 'sake', 'mirin', 'sesame oil', 'skinless pork belly', 'unsalted butter', 'slider buns', 'mayonnaise', 'english cucumbers'],
['whole chicken', 'white or black peppercorns', 'garlic', 'cilantro roots', 'light brown sugar', 'fish sauce', 'ground turmeric', 'kosher salt', 'ground coriander', 'ground cumin', 'ground cardamom', 'fresh ripe pineapple', 'distilled white vinegar or apple cider vinegar', 'garlic', 'red jalapeños or serrano chiles', 'granulated sugar', 'kosher salt', 'freshly ground white pepper', 'saffron threads, ground turmeric', 'granulated sugar', 'kosher salt', 'virgin coconut oil', 'vegetable oil', 'cilantro', 'jasmine rice, coconut rice'],
['egg yolks', 'all-purpose flour', 'almond flour, all-purpose flour', 'sugar', 'kosher salt', 'unsalted butter', 'whole milk', 'vanilla paste or, vanilla extract', 'egg yolks', 'sugar', 'cornstarch', 'kosher salt', 'unsalted butter', 'fresh blueberries', 'fresh blackberries', 'special equipment, springform pan or tart pan'],
['raw thai jasmine rice', 'coconut milk', 'water', 'granulated sugar', 'salt', 'coconut cream'],
['raw sesame seeds', 'lemon', 'peaches ', 'sugar', 'kosher salt', 'all-purpose flour', 'unsalted butter', 'ground cinnamon', 'tahini', 'sesame oil', 'cornstarch', 'vanilla ice cream'],
['hazelnuts', 'sugar', 'all-purpose flour', 'unsalted butter', 'eggs', 'vanilla extract', 'almond extract', 'tart shell', 'firm-ripe bosc or anjou pears', 'apricot preserves'],
['kosher salt', 'broccoli', 'whole-wheat penne pasta', 'extra-virgin olive oil', 'panko or freezer bread crumbs', 'oil-packed anchovy fillets', 'capers', 'garlic', 'red pepper flakes'],
['water', 'dashima', 'dried shiitake mushrooms', 'dried anchovies'],
['water', 'sour tamarind pulp or paste', 'fresh ginger', 'sugar', 'club soda, water'],
['saffron strands', 'sugar', 'green cardamom pods', 'ground sumac', 'club soda or water'],
['smooth peanut butter', 'light brown sugar', 'egg', 'all-purpose flour', 'bicarbonate of soda', 'fine sea salt', 'salted peanuts', 'unsalted butter', 'cocoa powder', 'caster, sugar', 'light brown sugar', 'fine sea salt', 'vanilla extract', 'eggs', 'all-purpose flour'],
['neutral cooking oil', 'onion', 'garlic', 'poblano chiles', 'corn tortillas', 'limes', 'bacon', 'medium shrimp', 'kosher salt', 'queso asadero cheese', 'limes', 'tangy red salsa'],
['cream cheese', 'kimchi', 'scallions', 'cilantro leaves', 'lime', 'kosher salt', 'country-style bread', 'chili oil and toasted white sesame seeds'],
['scallions', 'unsalted butter', 'ginger', 'tomatoes', 'white or regular soy sauce', 'kosher salt', 'toasted country-style bread, rice or pasta'],
['extra-virgin olive oil', 'garlic', 'cherry tomatoes', 'basil', 'red pepper flakes', 'kosher salt', 'sugar', 'casarecce or other medium-size pasta', 'parmesan'],
['sleeve, ritz crackers', 'raw peanuts or other nut or seed', 'light or dark brown sugar', 'diamond crystal or, morton kosher salt', 'unsalted butter', 'eggs', 'light or dark brown sugar', 'diamond crystal or, morton kosher salt', 'vanilla extract', 'natural peanut butter or other natural nut or seed butter', 'heavy cream', 'grape jelly', 'jam'],
['semi-pearled farro', 'kosher salt', 'eggs', 'watermelon radish or 6-8 red radishes', 'unseasoned rice vinegar', 'sugar', 'block extra-firm tofu', 'grapeseed or vegetable oil', 'fresh lime juice', 'gochujang', 'mayonnaise', 'soy sauce', 'sesame oil', 'zucchini or summer squash', 'raw vegetables', 'scallions', 'sugar snap peas', 'baby bok choy', 'cucumbers', 'pea shoots'],
['heirloom tomatoes', 'mixed cherry tomatoes', 'kosher salt', 'lemon', 'garlic', 'extra-virgin olive oil', 'zaatar', 'pita chips', 'honey', 'feta', 'basil leaves', 'mint leaves'],
['skin-on, chicken thighs', 'kosher salt', 'cherry tomatoes', 'harissa paste', 'red wine vinegar', 'oregano', 'feta, planks', 'crusty bread'],
['jalapeños', 'ginger', 'garlic', 'cilantro leaves', 'fresh lime juice', 'tahini', 'kosher salt', 'extra-virgin olive oil', 'garlic', 'chickpeas rinsed', 'corn', 'spice blend', 'kosher salt', 'tender greens', 'avocados', 'tomatoes', 'feta', 'cilantro', 'and lime wedges'],
['unsalted butter', 'raw pistachios', 'egg', 'light brown sugar', 'all-purpose flour', 'kosher salt', 'baking powder', 'eggs', 'light brown sugar', 'sour cream', 'vanilla bean paste or vanilla extract', 'ground cardamom', 'ground ginger', 'orange zest', 'kosher salt', 'all-purpose flour', 'stone fruit', 'coarse sugar', 'powdered sugar, cream', '" -diameter springform pan'],
['heirloom bananas or small plantains', 'crema or cre me frai che', 'crumbled queso fresco'],
['fresh chorizo', 'ground beef', 'tomato paste', 'cumin', 'onion', 'jalapeños', 'kosher salt', 'beefsteak tomatoes', 'corn kernels', 'cheddar cheese', 'eggs', 'unsalted butter', 'sour cream', 'cornmeal', 'baking powder'],
['garlic', 'pickled peppers', 'stone-ground or whole grain mustard', 'kosher salt', 'plum tomatoes', 'thick-cut bacon', 'crusty multigrain bread', 'slightly firm peaches']]


In [18]:
# Zero-shot prompting
#1: consistent results with link
prompt1="[INST] 1)Give me recipe name using "
instruction1= " 2) Give me the instructions 3)Give me the source URL [/INST]"
#4: consistent results with 1) Recipe name 2) Ingredients 3) Instructions 4) source url
prompt4="[INST] 1)Using these ingredients such as "
instruction4= " to find a 1)Recipe Name 2) Detailed instructions for the recipe 3) importantly Source URL.  [/INST]"
#6: consistent results with ingredient portions, instructions, source url
prompt6="[INST] Give one recipe with steps using "
instruction6= " Give source url. [/INST]"

# Chain of thoughts prompting
#7: consistent results 1) Recipe Name 2) Ingredient portions 3) Instructions 3) Source URL
prompt7="[INST] Give one named recipe with steps using "
instruction7= " Give source url. Please think step by step and answer fully. [/INST]"
#2: consistent results 1) Recipe Name 2) Instructions 3) Source URL
prompt2="[INST] 1)Give me recipe name using "
instruction2= " Without finding the recipe name just yet, think through this carefully and list briefly what you need to find when coming up with a recipe. For example you need to state, 1) Recipe Name 2) Detailed instructions for the recipe 3)Source URL.  [/INST]"
#3: consistent results 1) Recipe Name 2) Instructions 3) Source URL
prompt3="[INST] 1)Give me recipe name using "
instruction3= " Without finding the recipe name just yet, think through this carefully and list briefly what you need to find when coming up with a recipe. For example you need to state, 1) Recipe Name 2) Detailed instructions for the recipe 3) importantly Source URL.  [/INST]"
#5: consistent results with 1) Recipe name 2) Ingredients 3) Instructions 4)source url
prompt5="[INST] 1)Given these ingredients"
instruction5= " find the recipe name, the instructions to make this recipe and source URL. Do not immediately come up with the instructions lits, think through carefully and systematically state 1)Recipe Name 2) Detailed instructions for the recipe 3) importantly Source URL.  [/INST]"

# other CoT prompts we have tried. Result is consistent: 1) Recipe name 2) Ingredients portion 3) Instructions 4) source url
# prompts = [
#     f"[INST] Give one named recipe with steps using ingredients: {ing}. Give source url. [/INST]",
#     f"[INST] My ingredients are {ing}. Give 1) Recipe name, 2) source url. Please think step by step.[/INST]",
#     f"[INST] My ingredients are {ing}. Give 1) Recipe name, 2) source url of the recipe. Please think step by step.[/INST]"
#     f"[INST] Give 1) Recipe name, 2) source url using ingredients {ing}. Please think step by step.[/INST]",
#     f"[INST] Use the ingredients {ing}. Format will be Recipe: \nIngredients: \nSteps: \nSource url:[/INST]",
#     f"[INST] Use the ingredients {ing}. Retrieve the recipe from a valid source. An example of a valid source is 'https://www.allrecipes.com/recipe/83576/tofu-salad/'. Format is \nRecipe: \nIngredients: \nSteps: \nSource url:[/INST]"
# ]

Few Shot prompting: since the model is consistent with its results, we attempted few shot prompting to inform model to generate the correct link. 
1. example of correct url
2. tell it to regenerate the url, passing back its invalid url as an example not to do

In [39]:
import time

# 1. pass llm example of correct url
# ing = ", ".join(list[97])
prompt_fs = f"[INST] Use the ingredients "
instruction_fs =  " Some examples of a source url are 'https://www.allrecipes.com/recipe/83576/tofu-salad/'. Do not return me the example source url. Format is \nRecipe: \nIngredients: \nSteps: \nSource url: [/INST]"

# 2. check if url is invalid and return the llm the invalid url as example of what not to do
# for prompt in prompts:
#     # result is type list with a dict with one key 'generated_text'
#     result = llm(prompt)
#     valid_url = check_url_status(result[0]['generated_text'])
#     invalid_urls = []
#     while valid == 0:
#         invalid_urls.append(valid_url[1])
#         invalid_urls_str = ", ".join(invalid_urls)
#         print(invalid_urls)
#         result = llm(f"[INST] Use the ingredients {ing}. Retrieve the recipe from a valid source. Format is \nRecipe: \nIngredients: \nSteps: \nSource url: \nThese urls are invalid:{invalid_urls_str} [/INST]")
#         time.sleep(0.5)
#         valid_url = check_url_status(result[0]['generated_text'])
#     print("\n ----NEXT PREDICTION----")

In [40]:
final_prompts=[]
for ingredient in list:
    #print(ingredient)
    length_ing = len(ingredient)
    prompt=prompt_fs
    for j in range(length_ing):
        prompt +=ingredient[j]
        if j==length_ing-1:
            prompt+='.'
        else:
            prompt+=', '
    final_prompts.append(prompt)
final_prompt_eng = []
#Prompt engineering 
for instruction in final_prompts: 
  instruction+=instruction_fs
  final_prompt_eng += [instruction]
print(final_prompt_eng[0])

[INST] Use the ingredients whole chicken, kosher salt, acorn squash, sage, rosemary, unsalted butter, ground allspice, crushed red pepper flakes, freshly ground black pepper, white bread, apples, extra-virgin olive oil, red onion, apple cider vinegar, white miso, all-purpose flour, unsalted butter, dry white wine, unsalted chicken broth, white miso, kosher salt, freshly ground pepper. An example of a valid source is 'https://www.allrecipes.com/recipe/83576/tofu-salad/'. Do not return me the example source url. Format is 
Recipe: 
Ingredients: 
Steps: 
Source url: [/INST]


Results of few shot prompting
1. The model either returns an invalid url close to the example or the example itself. 
2. Simply not enough context nor time to let the model regenerate a url
Hence we tried RAG next as few shot is not sufficient. 

In [41]:
predictions = []
for question in final_prompt_eng[:3]:  # here we only use the first 100 for testing
    print(f"Question: ", question)
    #chain of thought prompt engineering
    #make big question into smaller questions
    #in first question
    result = llm(question)

    predictions.append(result)

Question:  [INST] Use the ingredients whole chicken, kosher salt, acorn squash, sage, rosemary, unsalted butter, ground allspice, crushed red pepper flakes, freshly ground black pepper, white bread, apples, extra-virgin olive oil, red onion, apple cider vinegar, white miso, all-purpose flour, unsalted butter, dry white wine, unsalted chicken broth, white miso, kosher salt, freshly ground pepper. An example of a valid source is 'https://www.allrecipes.com/recipe/83576/tofu-salad/'. Do not return me the example source url. Format is 
Recipe: 
Ingredients: 
Steps: 
Source url: [/INST]
1 [INST] Use the ingredients whole chicken, kosher salt, acorn squash, sage, rosemary, unsalted butter, ground allspice, crushed red pepper flakes, freshly ground black pepper, white bread, apples, extra-virgin olive oil, red onion, apple cider vinegar, white miso, all-purpose flour, unsalted butter, dry white wine, unsalted chicken broth, white miso, kosher salt, freshly ground pepper. An example of a valid

In [ ]:
print(predictions)

Bad prompts:
1. Give 1) Recipe name, 2) source url for ingredients {ing}. Please think step by step.
The model gave a link for each ingredient.

# # RAG

In [ ]:
import pandas as pd
file_path = "/kaggle/input/final-recipes/final_recipes.xlsx"
# Read the CSV file and handle the error by skipping the problematic row
# Read the Excel file into a pandas DataFrame
try:
    df = pd.read_excel(file_path)
    # Now 'df' contains the data from the Excel file
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")

In [ ]:
columns_to_remove= ["Unnamed: 0"]
df = df.drop(columns=columns_to_remove, axis=1)

In [ ]:
df

In [ ]:
import os
import urllib.request
import tarfile
import json
import pandas as pd
import numpy as np
from tqdm import tqdm


#libraries for text preprocessing
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

#libraries for keyword extraction with tf-idf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import coo_matrix

#libraries for reading and writing files
import pickle

#libraries for BM25
!pip install rank_bm25
from rank_bm25 import BM25Okapi

## Evaluation

### 1. BERT score

In [ ]:
import evaluate
import numpy as np

ans_title = df["Title"]
ans_ing = df["final_ing"]
num_qns_to_eval = 20

#if want recipe name put this in references=
ans_title = ans_title.values.tolist()[:num_qns_to_eval]

#if want ingredients put this in references=
ans_ing = ans_ing.values.tolist()[:num_qns_to_eval]

#if want recipe name + ingredients put this in references=
ans_ing_final = []
# to convert the string to list
for str_ing in ans_ing:
  ans_ing_final.append(str_ing[1:-1].replace("'", "").split(', '))

print(ans_title)
print(ans_ing_final)

for i in range(len(ans_title)):
  ans_ing_final[i].append(ans_title[i])

bertscore = evaluate.load("bertscore")
results = bertscore.compute(predictions=predictions, references=ans_ing_final, lang="en")
bert_s = results['precision']
bert_r = results['recall']
bert_f1 = results['f1']


print(f"BERTScore Precision: {np.mean(bert_s)}")
print(f"BERTScore Recall: {np.mean(bert_r)}")
print(f"BERTScore F1: {np.mean(bert_f1)}")

### 2. BLEU score

In [ ]:
# BLEU score
# will modify based on whether the model can give a consistent answer or not

bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=ans_ing_final)
print(results)

### 3. Custom scoring

In [ ]:
import requests
import time
import re
import random

# fake headers so I dont get blocked
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15'
]

def check_url_status(pred, score_weight=0.3):
    # score weight to be added to total
    
    if "http" not in pred:
        return 0
    url_match = re.search(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', pred)
    
    if url_match:
        url = url_match.group()
        try:
            headers = {'User-Agent': random.choice(user_agents)}
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                print(f"{url} returns 200 (OK).")
                return score_weight
            else:
                print(f"{url} returns 404 (Not Found) or invalid link.")
                return 0
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
        
    return 0

# --------------FOR TESTING, COMMENT OUT WHEN ACTUAL--------------
eg_pred_valid = "recipe blabla ing bla https://www.allrecipes.com/recipe/32217/pickled-sausage/"
eg_pred_invalid = "recipe blabla ing bla https://www.allrecipes.com/recipe/32217/poop-sausage/"
# Extract from predictions:
fake_pred = [eg_pred_valid, eg_pred_invalid] 

for url in fake_pred:
    bin_val = check_url_status(url)
    print(bin_val)
    time.sleep(0.5)
# --------------FOR TESTING, COMMENT OUT WHEN ACTUAL--------------

In [ ]:
# own grp eval. check if >=50% of items in the igredients+title list is in the answer, if yes total+=1.
# final score is total predictions with >=50% of items / total num of predictions

# we can adjust threshold based on model performance
threshold = 0.5
total = 0
count = len(predictions)
print(predictions)
print(ans_ing_final)

for i in range(len(predictions)):
  p_inside = 0
  total_items_per_set = len(ans_ing_final[i])
  
  for recipe_item in ans_ing_final[i]:
    inside = 1
    if recipe_item.lower() not in predictions[i].lower():
      inside = 0
    p_inside += inside

  url_score = check_url_status(predictions[i])
  
  # url_score weight adjustable in function. 
  p_final = (p_inside+url_score)/total_items_per_set
  
  if p_final >= 0.5:
    total += 1

print(total, count)
binary_score = total/count

print(binary_score)